In [1]:
import pandas as pd
import json
import topicvisexplorer
import importlib
import pickle
import random
from _topic_similarity_matrix import *
#from _topic_splitting_helpers import *
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['usernameremoved', 'link', 'remove', 'usernameremove', 'amp', 'linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])

name_column_text = 'texto_completo'
name_tokenizacion = 'text_cleaner'

In [2]:
# Get document relevant seed words
f = open('json_file_topic_splitting_test.json',) 
data = json.load(f) 
new_document_seeds_TopicA = data['new_document_seeds']['TopicA']
new_document_seeds_TopicB = data['new_document_seeds']['TopicB']
old_circle_positions = data['old_circle_positions']
current_number_of_topics = data['current_number_of_topics']
topic_id = data['topic_id'] # parte del 1
f.close()


In [4]:
importlib.reload(topicvisexplorer)
vis = topicvisexplorer.TopicVisExplorer("name")
vis.load_single_corpus_data("models_output/single_corpus_europe_cambridge_analytica_lda_mallet_with_word_embedding_final_dataset_user_study_topics_labeled.pkl")


Data loaded sucessfully


In [5]:
single_corpus_data = topicvisexplorer.single_corpus_data
optimization = True

In [6]:
#Import word embedding
word_embedding_model = single_corpus_data['word_embedding_model']
wordembedding = single_corpus_data['word_embedding_model']

In [7]:
#data for optimization

with open('data_for_optimization/list_relevant_documents_with_vectors_for_topic_splitting.pickle', 'rb') as handle:
    data_for_optimization = pickle.load(handle)            


In [35]:
data_for_optimization.columns

Index([               0,                1,                2,                3,
                      4,                5,                6,                7,
                      8,                9,               10, 'texto_completo',
         'text_cleaner',       '0_vector',       '1_vector',       '2_vector',
             '3_vector',       '4_vector',       '5_vector',       '6_vector',
             '7_vector',       '8_vector',       '9_vector',      '10_vector'],
      dtype='object')

In [36]:
#1.; Idea mandar parte del arreglo del lambdata!!

In [37]:
#Get list of relevant terms. Terms more relevant are going to appear first
PreparedData_dict_with_more_info = single_corpus_data['tinfo_collection']
list_terms_relevance = PreparedData_dict_with_more_info.loc[PreparedData_dict_with_more_info['Category'] == 'Topic'+str(topic_id)].sort_values(by='relevance', ascending=False)['Term'].tolist()


In [74]:
data_for_optimization['text_cleaner']

0         [meet, new, product, transparent, camhatch, we...
1         [week, trigger, see, lefty, trump, sadly, mayb...
2                                          [finest, remove]
3         [s, confident, privacy, protection, personal, ...
4                   [obscure, canadian, tech, firm, remove]
                                ...                        
111740    [call, startup, bring, value, industrial, sect...
111741    [mimic, disabled, people, sexually, abuse, wom...
111742    [make, upset, let, delete, awful, status, pop,...
111743                 [use, hashtag, ad, facebook, remove]
111744    [digital, age, datum, make, well, informed, de...
Name: text_cleaner, Length: 111745, dtype: object

In [82]:
data_for_optimization.head()

,0,1,2,3,4,5,6,7,8,9,...,1_vector,2_vector,3_vector,4_vector,5_vector,6_vector,7_vector,8_vector,9_vector,10_vector
0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,...,"[-0.005832044226394828, 0.004824629630888921, ...","[0.17334513068948354, 0.4751854061034919, -1.0...","[0.007541435136715791, -0.005965379836311513, ...","[-0.11085861998157043, -0.053958353831603745, ...","[-0.16557050217070923, 0.04593195003303663, 0....","[-8.961415625208247e-05, 0.0001998736659061251...","[-0.4008968984995249, 0.33226708981874253, -0....","[0.008453479881438852, -0.061913447329061455, ...","[-3.1126375603207634e-05, 0.000324685262853563...","[0.006710508750586541, 0.005146126403815288, -..."
1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,...,"[0.47801444855747377, -1.406873718365432, 0.90...","[-6.75392171274325e-05, -2.130824873347592e-05...","[-6.702237811850864e-05, -2.114604035341472e-0...","[-6.693600602147853e-05, -2.1092522047183593e-...","[-0.17950507357764423, -0.05090236161026951, 0...","[0.00011658700321959259, 2.6941313763018115e-0...","[-0.0003428991963687622, -0.000321200400094312...","[0.0010872332971416654, 0.008660244556040197, ...","[-0.00021789716439002405, 0.000529974763367135...","[-0.004681282343824478, 0.007711118507131687, ..."
2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,...,"[-7.068185425396223e-05, 0.0001059592759702354...","[-6.940999679727611e-05, 0.0001058115776686463...","[-5.7999792318241816e-05, 9.666439291322604e-0...","[-5.586606670249239e-05, 9.499314182903618e-05...","[-5.5158763558438295e-05, 9.45056090131402e-05...","[-8.213255051714441e-05, 0.0001150737152784131...","[-5.368500802660492e-05, 9.3279559223447e-05, ...","[-5.1279670230997e-05, 9.123190102400258e-05, ...","[-5.0732120058682995e-05, 9.091558968066238e-0...","[-4.9866681308685656e-05, 9.022851008921862e-0..."
3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,...,"[0.001956692503767954, -0.0014925585828677868,...","[0.7237029657366634, 0.7550907500003632, -0.63...","[0.48076796395451993, 0.25955325932636697, 0.3...","[0.0001122247154512479, 0.00018976365345224622...","[-0.011298434824226433, -0.2411481819126493, -...","[7.353772724627561e-05, -0.0004141479512327350...","[0.007881602803934129, 0.0034839859345083823, ...","[0.00011209843165715938, 0.0001891622232506051...","[-0.00028662316950089917, 0.000659989788346138...","[-0.008899104187946705, 0.010824778607911867, ..."
4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,...,"[0.07708563475614483, -0.08111963419287349, -0...","[0.027712605423914738, -0.015586473551593372, ...","[0.0025146853035380445, -0.0037733842618763447...","[9.155319560250064e-06, -0.0001351459086436079...","[9.159173913531049e-06, -0.0001351810369669692...","[0.01230773606908997, -0.01799366630439181, 0....","[9.164873006284324e-06, -0.0001356177135676262...","[0.000405963665514264, -0.00035627104989544023...","[9.143537766931331e-06, -0.0001351759810859221...","[9.136198116266314e-06, -0.0001350651427856064..."


In [83]:
'''
if optimization == True and int(topic_id)-1 in data_for_optimization.columns:
    list_relevant_documents = data_for_optimization.to_dict('records')
else:
    list_relevant_documents = random.sample(single_corpus_data['relevantDocumentsDict'],10000)
    list_relevant_documents = pd.DataFrame(list_relevant_documents).sort_values(int(topic_id)-1, ascending=False).reset_index()
    #the idea is do this only ONCE! and tenerlo precalculado para el user study
    print('cleaning sample fo text')
    list_relevant_documents[name_tokenizacion] = list_relevant_documents[name_column_text].apply(lambda x: text_cleaner(x))
    list_relevant_documents = list_relevant_documents.to_dict('records')
    print('cleaning documents seeds topic a')

    new_document_seeds_TopicA = pd.DataFrame(new_document_seeds_TopicA).reset_index()
    new_document_seeds_TopicA[name_tokenizacion] = new_document_seeds_TopicA[name_column_text].apply(lambda x: text_cleaner(x))
    new_document_seeds_TopicA = new_document_seeds_TopicA.to_dict('records')
    print('cleaning documents seeds topic B')

    new_document_seeds_TopicB = pd.DataFrame(new_document_seeds_TopicB).reset_index()
    new_document_seeds_TopicB[name_tokenizacion] = new_document_seeds_TopicB[name_column_text].apply(lambda x: text_cleaner(x))
    new_document_seeds_TopicB = new_document_seeds_TopicB.to_dict('records')
    print('getting new subtopics')
''';

In [69]:
#/////////////////////////////Parameters////////////////////////////
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['usernameremoved', 'link', 'remove', 'usernameremove', 'amp', 'linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])

name_column_text = 'texto_completo'
name_tokenizacion = 'text_cleaner'
#/////////////////////////////Parameters////////////////////////////




import json
import importlib
import topicvisexplorer
import pandas as pd
import warnings
from scipy import spatial

import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.models import LdaMulticore

import unidecode
import _prepare
import _prepare_single_topic
import gensim_helpers
import spacy
import re, numpy as np, pandas as pd

#libraries to tokenize text
from string import punctuation
from string import digits
from nltk.tokenize import TweetTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)





#this should be easy to change for users
punctuation+="¡¿<>'`"
punctuation+='"'
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

tknzr = TweetTokenizer()


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


#this should be easy to change for users
punctuation+="¡¿<>'`"
punctuation+='"'
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

tknzr = TweetTokenizer()
def sent_to_words(sentence):
    return tknzr.tokenize(sentence)
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    doc = nlp(" ".join(texts)) 
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
        
def text_cleaner(tweet):

    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()

    tweet = unidecode.unidecode(tweet)
    tweet = sent_to_words(tweet)

    tweet = remove_stopwords(tweet)

    new_tweet  = []
    for elem in tweet:
        if len(elem)>0:
            new_tweet.append(elem[0])
    tweet = lemmatization(new_tweet, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return tweet[0]


def getDocumentVector(text, wordembedding ,  list_terms_relevance):   

    document_vector = np.array([0.0]*300)# 300 dimensions
    
    for word in text:  
        #print('esta es una word', word)
        if word in list_terms_relevance:
            raking_word = float(list_terms_relevance.index(word)+1)
            if word in wordembedding: #if word in wordembedding.wv:
                #print("WORD FOUND", word, raking_word)
                document_vector+=wordembedding[word]/raking_word #aqui hay que ponderar
            else:
                pass
                #print("WARNING, Word not found:", word)    

    return document_vector


def get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,documents_class_A, documents_class_B, wordembedding):
    relevantDocumentsvector_class_A = 0.0
    list_documents_A = []
    for row in documents_class_A:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_A+= current_contribution*current_document_vector
        list_documents_A.append((current_contribution, current_text))
    
    relevantDocumentsvector_class_B= 0.0
    list_documents_B = []
    for row in documents_class_B:
        current_text = row[name_tokenizacion]
        current_contribution = row[str(int(topic_id) -1)]
        current_document_vector = getDocumentVector(current_text, wordembedding,  list_terms_relevance)    
        relevantDocumentsvector_class_B+= current_contribution*current_document_vector
        list_documents_B.append((current_contribution, current_text))

    return(relevantDocumentsvector_class_A, relevantDocumentsvector_class_B, list_documents_A, list_documents_B)



def fill_lists_documents_a_b(row, list_terms_relevance, vector_A, vector_B, documents_A, documents_B, most_relevant_documents_topic):

    current_contribution = row[int(topic_id)-1]
    current_text = row[name_tokenizacion]
    current_document_vector = getDocumentVector(current_text, wordembedding, list_terms_relevance).reshape(-1, 1)
    similarity_vectorA_currentvector = np.arccos(spatial.distance.cosine(vector_A, current_document_vector)-1) / np.pi
    similarity_vectorB_currentvector = np.arccos(spatial.distance.cosine(vector_B, current_document_vector)-1) / np.pi

    #I need this information to get the matrix of most relevant documents according to the similarity score

    #most_relevant_documents_topic.add((similarity_vectorA_currentvector,similarity_vectorB_currentvector,  current_contribution, row[name_column_text]))

    if similarity_vectorA_currentvector>= similarity_vectorB_currentvector:
        #append element to documentsA
        documents_A.append((current_contribution, row[name_tokenizacion]))
        most_relevant_documents_topic.add((similarity_vectorA_currentvector,0,  current_contribution, row[name_column_text]))

    else:
        documents_B.append((current_contribution, row[name_tokenizacion]))
        most_relevant_documents_topic.add((0,similarity_vectorB_currentvector,  current_contribution, row[name_column_text])) #QUIZAS LO CORRECTO Es que en vez de 0, sea 1-similarity_vectorB_currentvector



def create_two_list_of_documents(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    documents_A = []
    documents_B = []
    most_relevant_documents_topic = set()
    new_document_seeds_TopicA_df = pd.DataFrame(new_document_seeds_TopicA)
    new_document_seeds_TopicB_df = pd.DataFrame(new_document_seeds_TopicB)
    
    vector_A, vector_B, seeds_documents_A, seeds_documents_B = get_initial_document_vector_by_class(list_terms_relevance, topic_id, name_tokenizacion,new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)
    vector_A = vector_A.reshape(-1, 1)
    vector_B = vector_B.reshape(-1, 1)


    list_relevant_documents = pd.DataFrame(list_relevant_documents).sort_values(int(topic_id)-1, ascending=False).reset_index()
    
    print(' que es esta wea',list_relevant_documents.head())    
    list_relevant_documents.apply(lambda row:  fill_lists_documents_a_b(row, list_terms_relevance,vector_A, vector_B, documents_A, documents_B, most_relevant_documents_topic), axis=1)
    new_document_seeds_TopicA_df.apply(lambda row:  fill_lists_documents_a_b(row, list_terms_relevance,vector_A, vector_B, documents_A, documents_B, most_relevant_documents_topic), axis=1)
    new_document_seeds_TopicB_df.apply(lambda row:  fill_lists_documents_a_b(row, list_terms_relevance,vector_A, vector_B, documents_A, documents_B, most_relevant_documents_topic), axis=1)
    
    
    print('Documentos en A temp ', len(documents_A))
    print('Documentos en B temp ', len(documents_B))
    
        
    return (seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic)

        


def getCorpusDictionaryfromSentences(sentences):
    print('lda started')
    #data_words = list(sent_to_words(sentences))
    #data_ready = process_words(sentences)  # processed Text Data!
    # Create Dictionary
    id2word = corpora.Dictionary(sentences)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in sentences]
    # Build LDA model
    print('sendind corpus and dictionary')
    return (corpus, id2word)


def get_new_subtopics(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding):
    final_list_A = []
    final_list_B = []

    seeds_documents_A, seeds_documents_B, documents_A, documents_B, most_relevant_documents_topic = create_two_list_of_documents(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, wordembedding)
    #print('seeds documents a', len(seeds_documents_A))
    #print('seeds documents b', len(seeds_documents_B))
    #print('seeds documents a', len(documents_A))
    #print('seeds documents a', len(documents_B))
    
    print('tipo seeds', type(seeds_documents_A))
    print(seeds_documents_A)
    for contribution, text in seeds_documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in seeds_documents_B:
        final_list_B.extend([text]*int(contribution*100))

    for contribution, text in documents_A:
        final_list_A.extend([text]*int(contribution*100))
    for contribution, text in documents_B:
        final_list_B.extend([text]*int(contribution*100))
    
    freq_topic_A = len(final_list_A)/(len(final_list_A)+len(final_list_B))
    freq_topic_B = len(final_list_B)/(len(final_list_A)+len(final_list_B))
    
    return(getCorpusDictionaryfromSentences(final_list_A), getCorpusDictionaryfromSentences(final_list_B), most_relevant_documents_topic, freq_topic_A, freq_topic_B)

def update_topic_term_dists(row, total_frequency):
    row['topic_term_dists'] = row['term_frequency']/total_frequency
    return row

    
def extract_data_without_topic_model(corpus, dictionary):


    topic_model = None
    if not gensim.matutils.ismatrix(corpus):
        corpus_csc = gensim.matutils.corpus2csc(corpus, num_terms=len(dictionary))
    else:
        corpus_csc = corpus
        # Need corpus to be a streaming gensim list corpus for len and inference functions below:
        corpus = gensim.matutils.Sparse2Corpus(corpus_csc)

    vocab = list(dictionary.token2id.keys())
    
    beta = 0.01
    fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)
    term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
    term_freqs[term_freqs == 0] = beta

    assert term_freqs.shape[0] == len(dictionary),\
        'Term frequencies and dictionary have different shape {} != {}'.format(
        term_freqs.shape[0], len(dictionary))



    topic_term_dists = term_freqs/term_freqs.sum(axis=0) # esta bien esto! 
    return {'topic_term_dists': topic_term_dists, 'vocab': vocab, 'term_frequency': term_freqs}


def change_frequency_on_prepared_data(row, new_subtopic_df, total_sum_frequency_corpus):
    current_term = row['Term']
    current_total = row['Total']
    new_subtopic_df = pd.DataFrame(new_subtopic_df)
    current_total_new_subtopic_df = new_subtopic_df['term_frequency'].sum()
    if current_term in list(new_subtopic_df['vocab']) and current_total>0:
        new_subtopic_df = pd.DataFrame(new_subtopic_df)
        old_freq = row['Freq']
        new_prob = float(new_subtopic_df.loc[new_subtopic_df['vocab'] == current_term]['topic_term_dists'])
        current_frequency = float(new_subtopic_df.loc[new_subtopic_df['vocab'] == current_term]['term_frequency'])

        row['Freq'] = new_prob*row['Total']
        row['logprob'] = np.log(new_prob)
        #row['loglift'] = np.log(new_prob/(current_total/total_sum_frequency_corpus))     
        row['loglift'] = np.log(new_prob/(current_frequency/current_total_new_subtopic_df))                 

    else:
        row['Freq'] = 0    
        row['logprob'] = 0
        row['loglift'] = 0  
    return row
    


#Ojo, la frecuencia a actualizar sera del primer parametro q se le pase a la funcion, el q uno llava data model a
#en algun momento ahbra que intercambiar, data model a debe ser data model b. 
def update_current_freq_and_total_freq_on_prepared_data(row, data_model_A_df, data_model_B_df, list_terms_A, list_terms_B, total_sum_frequency_corpus):
    current_term = row['Term']
    if(current_term in list_terms_A):
        term_frequency_A = float(data_model_A_df.loc[data_model_A_df['vocab']==current_term]['term_frequency'])
    else:
        term_frequency_A = 0
    if(current_term in list_terms_B):
        term_frequency_B = float(data_model_B_df.loc[data_model_B_df['vocab']==current_term]['term_frequency'])
    else:
        term_frequency_B = 0
    row['Total'] = term_frequency_A+term_frequency_B
    row['Freq'] = term_frequency_A
    if(current_term in list_terms_A):
        new_prob = float(data_model_A_df.loc[data_model_A_df['vocab'] == current_term]['topic_term_dists'])
        row['logprob'] = np.log(new_prob)
        row['loglift'] = np.log(new_prob/((term_frequency_A+term_frequency_B)/total_sum_frequency_corpus))
    else:
        row['logprob'] = 0
        row['loglift'] = 0 #auqnue la verdad en evz de cero, creo que el valor debiese ser - inf
    return row


In [70]:

results  = get_new_subtopics(list_terms_relevance, list_relevant_documents, topic_id, name_tokenizacion,name_column_text, new_document_seeds_TopicA, new_document_seeds_TopicB, word_embedding_model)


 que es esta wea    level_0  index         0         1         2         3         4         5  \
0        0      2  0.114846  0.088000  0.072150  0.072150  0.103749  0.072150   
1        1      1  0.102694  0.084175  0.121197  0.084175  0.084175  0.084175   
2        2      0  0.090909  0.074516  0.074516  0.074519  0.074876  0.148599   
3        3      4  0.090879  0.079745  0.138139  0.091633  0.085401  0.079745   
4        4      7  0.090731  0.074592  0.074516  0.085725  0.074538  0.075877   

          6         7         8         9        10  \
0  0.072150  0.104043  0.124479  0.092942  0.083340   
1  0.084175  0.102703  0.084175  0.084175  0.084180   
2  0.091246  0.098453  0.106942  0.074591  0.090834   
3  0.079745  0.079757  0.079745  0.097289  0.097923   
4  0.090909  0.074516  0.123719  0.106098  0.128781   

                                      texto_completo  \
0  The trouble is they get away with it. If comme...   
1  <usernameremoved>   #Google Takes Aim at USA B... 

In [42]:




model_topic_A, model_topic_B, most_relevant_documents_topic, freq_topic_A, freq_topic_B = results
#  CREAR PICKLEEE!!! CON ESTA DATAAA!!!
with open('models_output/testing_spliting_models_topic_A_B.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=4) #protocol 4 is compatible with python 3.6+
    print("Results for topic splitting has been saved")
print('Geeting new term-topic distributions')
# Get new term-topic distributions in the new subtopics
#get new distribution of terms, topic A
corpus_topic_A, dictionary_topic_A = model_topic_A
data_model_A = extract_data_without_topic_model(corpus_topic_A, dictionary_topic_A)


corpus_topic_B, dictionary_topic_B = model_topic_B
data_model_B = extract_data_without_topic_model(corpus_topic_B, dictionary_topic_B)
print('actualizando el modelo A and B')
#filtrar por terminos que si aparezcan en lists terms relevance
df_temp = pd.DataFrame(data_model_A)
df_temp = df_temp[df_temp['vocab'].isin(list_terms_relevance)]
data_model_A = df_temp.to_dict()
df_temp = pd.DataFrame(data_model_B)
df_temp = df_temp[df_temp['vocab'].isin(list_terms_relevance)]
data_model_B = df_temp.to_dict()




#Get most relevant documents
print('Getting most relevant documents')
new_dict = dict()
#set columns of the new subtopics to NaN values
df = pd.DataFrame(single_corpus_data['relevantDocumentsDict'])

df[int(topic_id-1)]= 0.0
df[current_number_of_topics]= 0.0

for row in most_relevant_documents_topic:

    contribution_to_topic_a = row[0]
    contribution_to_topic_b = row[1]
    indexs = df.index[df['texto_completo'] == row[-1]].tolist()
    if len(indexs)<1:
        print('Error, text not found')
    #set final contribution to topic a, is contribution to topic_a multiply by the previous contribuiton
    df.loc[indexs,int(topic_id-1)] = contribution_to_topic_a
    df.loc[indexs,current_number_of_topics] = contribution_to_topic_b


#order columns
intList=sorted([i for i in df.columns.values if type(i) is int])
strList=sorted([i for i in df.columns.values if type(i) is str])
new_order = intList+strList
df = df[new_order]
single_corpus_data['relevantDocumentsDict'] = df.to_dict('records')

new_dict['relevantDocumentsDict_fromPython'] =json.dumps( single_corpus_data['relevantDocumentsDict'])

#Get prepared data
print('Getting new prepared data')
temp = single_corpus_data['PreparedDataObtained']

#update MdsDat
#add temporal coordinates. We are going to change these later with the new topic similarity metric.
temp['mdsDat']['x'].append(temp['mdsDat']['x'][topic_id-1])
temp['mdsDat']['y'].append(temp['mdsDat']['y'][topic_id-1])
temp['mdsDat']['topics'].append(len(temp['mdsDat']['topics'])+1)
temp['mdsDat']['cluster'].append(temp['mdsDat']['cluster'][topic_id-1])
#update the frequency of the topic
old_frequency = temp['mdsDat']['Freq'][topic_id-1]
temp['mdsDat']['Freq'][topic_id-1] = old_frequency*freq_topic_A
temp['mdsDat']['Freq'].append(old_frequency*freq_topic_B)
#Update topic.order
temp['topic.order'].append(len(temp['topic.order'])+1)

temp_tinfo_df = pd.DataFrame(temp[ 'tinfo'])
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].sort_values(by=['Freq'], ascending=False)
temp_tinfo_df = pd.DataFrame(temp[ 'tinfo'])

data_model_A_df = pd.DataFrame(data_model_A)
data_model_B_df = pd.DataFrame(data_model_B)
total_sum_frequency_corpus = data_model_A_df['term_frequency'].sum()+data_model_B_df['term_frequency'].sum()
list_terms_A = list(data_model_A_df['vocab'])
list_terms_B = list(data_model_B_df['vocab'])


temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)] = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)].apply(lambda row:  update_current_freq_and_total_freq_on_prepared_data(row, data_model_A_df,data_model_B_df, list_terms_A, list_terms_B,total_sum_frequency_corpus), axis=1)


#copy values for the new subtopic b
temp2 = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(topic_id)]
temp2.Category = 'Topic'+str(current_number_of_topics+1) 
temp_tinfo_df = temp_tinfo_df.append(temp2, ignore_index=True)

#update those values with the current terms probability\
temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(current_number_of_topics+1)] = temp_tinfo_df[temp_tinfo_df.Category == 'Topic'+str(current_number_of_topics+1)].apply(lambda row:  update_current_freq_and_total_freq_on_prepared_data(row, data_model_B_df,data_model_A_df, list_terms_B, list_terms_A,total_sum_frequency_corpus), axis=1)


#save the new tinfo
temp_tinfo_df.reset_index(drop=True, inplace=True)
temp[ 'tinfo']  = temp_tinfo_df.to_dict(orient='list')



print('New number of topics')

#Get new topic similarity matrix
print('Getting new topic similarity matrix')
newClass = TopicVisExplorer("name") #dejar esta en el codigo final
word_embedding_model = single_corpus_data['word_embedding_model']
topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6
print('Calculando topic similarity metrix')


lda_model = single_corpus_data['lda_model']
corpus = single_corpus_data['corpus']
id2word = single_corpus_data['id2word']
matrix_documents_topic_contribution = pd.DataFrame(single_corpus_data['relevantDocumentsDict'])


new_topic_similarity_matrix = newClass.calculate_topic_similarity_on_single_corpus_for_topic_splitting(current_number_of_topics, word_embedding_model, lda_model, corpus, id2word, matrix_documents_topic_contribution,topn_terms, topk_documents, relevance_lambda)
single_corpus_data['topic_similarity_matrix'] = new_topic_similarity_matrix
print('Topic similarity matrix has been calculated')



old_circle_positions = json_file['old_circle_positions']

for omega in old_circle_positions.keys():
    old_circle_positions[omega].append(old_circle_positions[omega][topic_id-1])


print('Calculating new circle positions with procrustes')

new_circle_positions = get_circle_positions_from_old_matrix(old_circle_positions, new_topic_similarity_matrix )
print('json new circle,. estas son las keys', json.loads(new_circle_positions).keys())
print('primer arreglo', json.loads(new_circle_positions)['0.0'])

single_corpus_data['new_circle_positions'] = new_circle_positions

print('------- falta calcular el nuevo topic orderingX')                 
topic_order =  single_corpus_data['topic.order']


#visualizar neuvos resultados

PreparedDataObtained = single_corpus_data['PreparedDataObtained'] 

#Return results in a dictionary


new_dict['new_circle_positions'] = single_corpus_data['new_circle_positions'] 

data = [single_corpus_data['PreparedDataObtained']]
data_json_format = []
for elem in data:
    elem = js.dumps(elem, cls=NumPyEncoder)
    data_json_format.append(elem)

new_dict['PreparedDataObtained_fromPython'] = js.loads(data_json_format[0])


#The following line is necessary to delete inf and nan values that javascript JSON.parse cant process
new_dict['PreparedDataObtained_fromPython']['tinfo'] = pd.DataFrame(new_dict['PreparedDataObtained_fromPython']['tinfo']).replace([np.inf, -np.inf, np.nan], 0).to_dict()

Results for topic splitting has been saved
Geeting new term-topic distributions
actualizando el modelo A and B
Getting most relevant documents
Getting new prepared data


KeyboardInterrupt: 